<a href="https://colab.research.google.com/github/sohyunwriter/ai-tech-archive/blob/master/stats_cal_ver0_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# dataset을 넣으면 기술통계를 산출하는 프로그램   

3번째 셀의 datafile과 outputfileName만 수정하셔서 쓰시면 됩니다   

input : 기술통계를 산출하고 싶은 데이터파일 (.xlsx, .csv)       
outputset : 기술통계 결과 csv 파일 (.csv)    


In [1]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
from collections import Counter
warnings.filterwarnings('ignore')
%matplotlib inline

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
datafile = "C:\\Users\\lsglobal\\Documents\\lse\\scr_dataset_0.xlsx"
#datafile = r"C:\Users\lsglobal\Documents\lsc\data\SCR_DataMart_111_1_ver0.3_20200724.csv"
ouputfileName = "scr_summary_happy.csv"

In [3]:
# read file and make df
def knowFileExt(datafile):
    return datafile.split(".")[-1]

def file2df(datafile):
    if (knowFileExt(datafile) == 'xlsx'):
        df = pd.read_excel(datafile)
    elif (knowFileExt(datafile) == 'csv'):
        df = pd.read_csv(datafile, encoding='utf-8')
    else:
        raise Exception  # raise error
    return df

def float2int(df, ms):  # float, int 구분
    for column in df.columns:
        if (df[column].dtypes != 'O'):
            if np.array_equal(df[column].dropna(), df[column].dropna().astype(int)):  # int values
                #df[column] = df[column].astype(int)
                ms.loc[column].type = np.arange(10, dtype='i').dtype
    return ms

def makeMS(df):  # 명세서 만들기
    ms = df.dtypes.to_frame(name="type")
    name = []
    for i in range(len(ms)):
        if (df[ms.iloc[i].name].dtypes == 'O'):
            name.append("명목")
        else:
            name.append("비율")
    ms["name"] = name
    return ms


def countUnique(df):  # 유일한 값 개수
    unique_list = []
    for i in range(len(df.columns)):
        unique_list.append(len(df.iloc[:, i].unique()))

    unique_series = pd.Series(unique_list, index=df.columns, name='유일한 값 개수')
    return unique_series


def multipleMostCommon(df, columnName):  # 열별로 최빈값 알아내기
    commonList = Counter(df[columnName]).most_common()
    mode_list = []
    modeNum = commonList[0][1]
    for i in range(len(commonList)):
        if commonList[i][1] < modeNum:
            break
        elif commonList[i][1] == modeNum:
            mode_list.append(commonList[i][0])
    return mode_list


def knowMostCommon(df):  # 최빈값
    modelist = []
    for i in range(len(df.columns)):
        modelist.append(multipleMostCommon(df, df.columns[i]))
    mode_series = pd.Series(modelist, index=df.columns, name='최빈값')
    return mode_series


def know_numericOrNon(ms, num):
    numericList = []
    nonNumericList = []
    for i in range(len(ms)):
        if ms.iloc[i].type != 'O':
            numericList.append(ms.iloc[i].name)
        else:
            nonNumericList.append(ms.iloc[i].name)
    if (num == 0):
        return numericList
    elif (num == 1):
        return nonNumericList
    else:
        return

def ds(df, ms):
    summary = df.describe()  # 1사분위수, 2사분위수, 3사분위수, 중앙값, 평균
    colName_series = ms.loc[:, 'name'].rename("구분").astype(str)
    count_series = df.count().rename("데이터 수").astype(str)
    unique_series = countUnique(df).rename("유일한 값 개수").astype(str)
    null_series = df.isnull().sum().rename("결측치 수").astype(str)
    mean_series = summary.loc['mean', :].rename("평균")
    median_series = summary.loc['50%'].rename("중앙값")
    mode_series = knowMostCommon(df).rename("최빈값")
    max_series = df.max(numeric_only=True).rename("최댓값")
    min_series = df.min(numeric_only=True).rename("최솟값")
    range_series = (max_series - min_series).rename("범위")
    per1_series = summary.loc['25%'].rename("1사분위수")
    per2_series = summary.loc['50%'].rename("2사분위수")
    per3_series = summary.loc['75%'].rename("3사분위수")
    std_series = summary.loc['std'].rename("표준편차")
    kurtosis_series = df.kurtosis().rename("첨도")
    skew_series = df.skew().rename("왜도")
    final = pd.concat([colName_series, count_series, unique_series, null_series, mean_series, \
                       median_series, mode_series, max_series, min_series, range_series, per1_series, \
                       per2_series, per3_series, std_series, kurtosis_series, skew_series], axis=1, ignore_index=False).T
    final.fillna("해당 없음", inplace=True)
    return final

    """
    def run():
    df = file2df(datafile)  # dataframe 읽기
    ms = float2int(df, makeMS(df)) # 명세서 만들기
    final = ds(df, ms) # 기술통계 산출
    final = final[df.columns] # 컬럼 정렬
    final.to_csv(outputfile, encoding='utf-8-sig')


    if __name__ == "__main__":
      # execute only if run as a script
      run()
    """

In [4]:
df = pd.DataFrame({"주조속도": [200, 400, 300, 400, 500], "원산지": ["미국", "미국", "한국", "한국", "중국"], "온도": [20.0, 40.0, 35.0, 35.0, 40.0]})

In [5]:
df

,주조속도,원산지,온도
0,200,미국,20.0
1,400,미국,40.0
2,300,한국,35.0
3,400,한국,35.0
4,500,중국,40.0


In [7]:
ms = float2int(df, makeMS(df))

In [9]:
ms

,type,name
주조속도,int32,비율
원산지,object,명목
온도,int32,비율


In [8]:
final = ds(df, ms)

In [10]:
final

,주조속도,원산지,온도
구분,비율,명목,비율
데이터 수,5,5,5
유일한 값 개수,4,3,3
결측치 수,0,0,0
평균,360,해당 없음,34
중앙값,400,해당 없음,35
최빈값,[400],"[미국, 한국]","[40.0, 35.0]"
최댓값,500,해당 없음,40
최솟값,200,해당 없음,20
범위,300,해당 없음,20


In [ ]:
# int 변환, 반올림, 콤마 -> 코드 추가 필요 

In [ ]:
final.to_csv(ouputfileName, encoding='utf-8-sig')